# Data Masters Case: Modelo de Classificação

Felipe Viacava -- São Paulo, ago/2023

O presente documento consiste no treinamento e avaliação de modelos de classificação como parte da solução do Case "Data Masters - Cientista de Dados" do Santander Brasil. O objetivo é treinar um modelo de classificação de clientes possivelmente insatisfeitos com o banco para selecionar o público alvo de uma campanha de retenção.

Além das premissas estabelecidas na etapa de exploração dos dados, assumimos que:
- Buscamos um modelo de alto poder preditivo e não um modelo para inferência;
- Para cada cliente marcado como insatisfeito (_predicted positive_) pelo modelo, temos um custo de R$10,00;
- Destes, para os verdadeiramente insatisfeitos (_true positives_), temos um retorno de R$100,00;
- O melhor modelo é aquele que maximiza o retorno sobre a campanha de retenção, ou seja, a soma de:
    - _False Positives_, que geram custo R$10,00 por observação;
    - _True Positives_, que geram lucro de R$90,00 por observação;
    - _False Negatives_, que não geram lucro nem custo por observação;
    - _True Negatives_, que não geram lucro nem custo por observação.

As premissas acima surgem da interpretação do enunciado do problema. Outras premissas que nortearam o desenvolvimento dos modelos são:
- O código deve ser limpo, conciso e reprodutível;
- A performance dos modelos deve ser comparada sobre um conjunto de dados nunca antes visto, prevenindo _data leakage_. Este conjunto foi separado antes da análise exploratória, simulando o que seria o acompanhamento da performance de um modelo após sua implementação;
- Uso de transformadores em _pipelines_ para qualquer tipo de processamento nos dados, garantindo fácil implementação e reprodutibilidade;
- Todos os modelos testados devem obrigatoriamente passar pelos mesmos passos, sendo estes:
    - Divisão dos dados entre treino e validação, estratificado por TARGET;
    - _GridSearchCV_ para hiperparametrização maximizando a _AUC_ (conjunto de treino);
    - Retreino após a escolha dos melhores hiperparâmetros (conjunto de treino);
    - Ajuste do corte de classificação maximizando a função $\frac{90tp - 10fp}{n}$ (conjunto de validação);
    - Retreino após a escolha do melhor corte de classificação e melhores hiperparâmetros (conjunto de treino + validação);
    - Avaliação e comparação no contexto de negócios (conjunto de testes).

## Bibliotecas

In [1]:
# --- Data Exploration and Viz --- #
import pandas as pd

# --- Custom Pipelines --- #
from resources.prep import build_prep, build_prep_nan

# --- Models --- #
import pickle
from resources.train_evaluate import TrainEvaluate

## Leitura dos dados

In [2]:
test = pd.read_csv("data/test.csv")

## Modelagem

### Pipeline base

Este pipeline segue os passos sugeridos durante a análise exploratória.

In [3]:
prep = build_prep()
prep

Pipeline(steps=[('DropConstantColumns', DropConstantColumns(also=['ID'])),
                ('DropDuplicateColumns', DropDuplicateColumns()),
                ('NoneZeroCountSaldo', AddNonZeroCount(prefix='saldo')),
                ('SumSaldo', CustomSum(prefix='saldo')),
                ('NoneZeroCountImp', AddNonZeroCount(prefix='imp')),
                ('SumImp', CustomSum(prefix='imp')),
                ('ImputeNanDelta',
                 CustomImputer(prefix='delta', to...
                ('NonZeroCountDelta', AddNonZeroCount(prefix='delta')),
                ('SumDelta', CustomSum(prefix='delta')),
                ('NonZeroContInd', AddNonZeroCount(prefix='ind')),
                ('NonZeroCountNum', AddNonZeroCount(prefix='num')),
                ('SumNum', CustomSum(prefix='num')),
                ('ImputeNanVar3',
                 CustomImputer(prefix='var3', to_replace=-999999)),
                ('CustomEncoderVar36', CustomEncoder(colname='var36')),
                ('CustomEncoderVar21', CustomEncoder(colname='var21'))])

### Preenchendo NaN

Diferentemente de outros algoritmos baseados em árvores de decisão do _sklearn_, o algoritmo _RandomForestClassifier_ não tem uma estratégia definida para lidar com dados faltantes. Assim, ao final do pipeline anterior, adicionamos uma última etapa que preenche os valores faltantes com a mediana da coluna. Foi escolhida a mediana pois:
- As variáveis de prefixo "Delta" (contínua) são esparsas e a suas médias seriam muito afetadas por outliers;
- Assumiu-se a variável "var3" como variável de contagem (discreta), não fazendo sentido usar a média para preenchimento;
- Como o dataset é composto por muitas variáveis e estas não foram analisadas individualmente, optou-se por não ajustar as escalas das variáveis e testar apenas modelos baseados em árvores. Assim, não faria sentido utilizar o _KNNImputer_ nesta versão da solução do _case_, e pode ser um ponto para melhorar em futuras entregas -- assim como o teste de modelos lineares.

In [4]:
prep_nan = build_prep_nan()
prep_nan

Pipeline(steps=[('prep',
                 Pipeline(steps=[('DropConstantColumns',
                                  DropConstantColumns(also=['ID'])),
                                 ('DropDuplicateColumns',
                                  DropDuplicateColumns()),
                                 ('NoneZeroCountSaldo',
                                  AddNonZeroCount(prefix='saldo')),
                                 ('SumSaldo', CustomSum(prefix='saldo')),
                                 ('NoneZeroCountImp',
                                  AddNonZeroCount(prefix='imp')),
                                 ('SumImp', CustomSum(prefix='imp')),
                                 ('ImputeNanDelta',
                                  CustomI...
                                 ('NonZeroContInd',
                                  AddNonZeroCount(prefix='ind')),
                                 ('NonZeroCountNum',
                                  AddNonZeroCount(prefix='num')),
                                 ('SumNum', CustomSum(prefix='num')),
                                 ('ImputeNanVar3',
                                  CustomImputer(prefix='var3',
                                                to_replace=-999999)),
                                 ('CustomEncoderVar36',
                                  CustomEncoder(colname='var36')),
                                 ('CustomEncoderVar21',
                                  CustomEncoder(colname='var21'))])),
                ('nan', SimpleImputer(strategy='median'))])

### Random Forest

In [5]:
with open("models/rf.pkl", "rb") as f:
    rf = pickle.load(f)

rf = rf.evaluate(test)

In [6]:
rf.best_model_

Pipeline(steps=[('preprocessor',
                 Pipeline(steps=[('prep',
                                  Pipeline(steps=[('DropConstantColumns',
                                                   DropConstantColumns(also=['ID'])),
                                                  ('DropDuplicateColumns',
                                                   DropDuplicateColumns()),
                                                  ('NoneZeroCountSaldo',
                                                   AddNonZeroCount(prefix='saldo')),
                                                  ('SumSaldo',
                                                   CustomSum(prefix='saldo')),
                                                  ('NoneZeroCountImp',
                                                   AddNonZeroCount(prefix='imp')),
                                                  ('SumImp',
                                                   CustomSum(prefix='...
                                                   CustomSum(prefix='num')),
                                                  ('ImputeNanVar3',
                                                   CustomImputer(prefix='var3',
                                                                 to_replace=-999999)),
                                                  ('CustomEncoderVar36',
                                                   CustomEncoder(colname='var36')),
                                                  ('CustomEncoderVar21',
                                                   CustomEncoder(colname='var21'))])),
                                 ('nan', SimpleImputer(strategy='median'))])),
                ('classifier',
                 RandomForestClassifier(class_weight='balanced', max_depth=8,
                                        max_features=64, n_estimators=500,
                                        random_state=42))])

## Histogram Gradient Boosting

In [7]:
with open("models/hgb.pkl", "rb") as f:
    hgb = pickle.load(f)

hgb = hgb.evaluate(test)

In [8]:
hgb.best_model_

Pipeline(steps=[('preprocessor',
                 Pipeline(steps=[('DropConstantColumns',
                                  DropConstantColumns(also=['ID'])),
                                 ('DropDuplicateColumns',
                                  DropDuplicateColumns()),
                                 ('NoneZeroCountSaldo',
                                  AddNonZeroCount(prefix='saldo')),
                                 ('SumSaldo', CustomSum(prefix='saldo')),
                                 ('NoneZeroCountImp',
                                  AddNonZeroCount(prefix='imp')),
                                 ('SumImp', CustomSum(prefix='imp')),
                                 ('ImputeNanDelta'...
                                 ('SumNum', CustomSum(prefix='num')),
                                 ('ImputeNanVar3',
                                  CustomImputer(prefix='var3',
                                                to_replace=-999999)),
                                 ('CustomEncoderVar36',
                                  CustomEncoder(colname='var36')),
                                 ('CustomEncoderVar21',
                                  CustomEncoder(colname='var21'))])),
                ('classifier',
                 HistGradientBoostingClassifier(categorical_features=['var36',
                                                                      'var21'],
                                                class_weight='balanced',
                                                max_depth=4, max_iter=50,
                                                random_state=42))])

## Comparação

In [9]:
business_results = pd.DataFrame(
    data = [
        rf.business_metrics,
        hgb.business_metrics
    ],
    index = [
        "Random Forest",
        "Histogram-based Gradient Boosting"
    ]
)

business_results.transpose()

,Random Forest,Histogram-based Gradient Boosting
Profit (Total),13930.000000,14340.000000
Profit (per Customer),0.916206,0.943173
True Positive Profit (Total),32130.000000,32850.000000
True Positive Profit (per Customer),2.113260,2.160616
False Positive Loss (Total),18200.000000,18510.000000
False Positive Loss (per Customer),1.197053,1.217443
False Negative Potential Profit Loss (Total),22050.000000,21330.000000
False Negative Potential Profit Loss (per Customer),1.450276,1.402920
True Negative Loss Prevention (Total),127820.000000,127510.000000
True Negative Loss Prevention (per Customer),8.406998,8.386609


In [10]:
business_results = pd.DataFrame(
    data = [
        rf.classification_metrics,
        hgb.classification_metrics
    ],
    index = [
        "Random Forest",
        "Histogram-based Gradient Boosting"
    ]
)

business_results.transpose()

,Random Forest,Histogram-based Gradient Boosting
Classification Threshold,0.616703,0.671201
ROC AUC,0.843350,0.850533
Precision,0.163987,0.164711
Recall,0.593023,0.606312
F1,0.256927,0.259049
Accuracy,0.864180,0.862668


## Variáveis mais importantes do modelo campeão

In [11]:
feature_importances = hgb \
    .get_feature_importances(test) \
    .reset_index(drop=True)

In [12]:
feature_importances[feature_importances['Importance'] > 0]

,Feature,Importance
0,var15,26396.666667
1,saldo_var30,13873.000000
2,saldo_medio_var5_ult1,1415.000000
3,var38,1307.000000
4,saldo_var5,1225.333333
5,sum_of_num,1131.666667
6,num_var45_hace3,1060.333333
7,saldo_medio_var5_hace2,602.666667
8,saldo_medio_var5_hace3,555.000000
9,num_var22_ult3,494.666667


In [13]:
feature_importances \
    .where(
        lambda ldf:
        ldf["Feature"].apply(
            lambda row:
            (
                row.startswith("none_")
                | row.startswith("sum_")
                | row.startswith("non_")
            )
        )
    ) \
    .dropna()

,Feature,Importance
5,sum_of_num,1131.666667
15,non_zero_count_num,164.666667
85,non_zero_count_delta,0.000000
125,none_count_delta,0.000000
277,non_zero_count_imp,-1.666667
279,non_zero_count_saldo,-3.666667
294,sum_of_delta,-51.333333
299,sum_of_imp,-73.333333
305,sum_of_saldo,-147.000000
307,non_zero_count_ind,-271.333333
